In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [5]:
df = pd.read_pickle('emotions_dataset.pkl')


In [38]:
df.head()

,Imagen,Imagen con zoom,Hitos faciales,Etiquetas,Hitos faciales válidos
0,"[[[165, 165, 162], [165, 166, 161], [165, 166,...","[[160, 160, 160, 160, 160, 160, 160, 160, 160,...","[[[62, 83], [62, 88], [62, 92], [63, 97], [64,...",bored,"[[[62, 83], [62, 88], [62, 92], [63, 97], [64,..."
1,"[[[164, 166, 161], [165, 167, 162], [164, 166,...","[[160, 160, 160, 160, 160, 160, 160, 160, 160,...","[[[62, 83], [62, 87], [63, 92], [63, 97], [64,...",bored,"[[[62, 83], [62, 87], [63, 92], [63, 97], [64,..."
2,"[[[165, 165, 163], [165, 165, 163], [164, 165,...","[[159, 159, 159, 159, 159, 159, 159, 159, 159,...","[[[63, 82], [63, 87], [63, 91], [63, 96], [65,...",bored,"[[[63, 82], [63, 87], [63, 91], [63, 96], [65,..."
3,"[[[165, 166, 162], [165, 165, 163], [165, 165,...","[[159, 159, 159, 159, 159, 159, 159, 159, 159,...","[[[63, 83], [63, 87], [63, 92], [63, 97], [64,...",bored,"[[[63, 83], [63, 87], [63, 92], [63, 97], [64,..."
4,"[[[165, 166, 161], [166, 167, 162], [165, 167,...","[[160, 160, 160, 160, 160, 160, 160, 160, 160,...","[[[63, 82], [63, 87], [63, 91], [63, 96], [65,...",bored,"[[[63, 82], [63, 87], [63, 91], [63, 96], [65,..."


In [39]:
print(df['Imagen con zoom'].values[0])

[[160 160 160 ... 161 161 161]
 [160 160 160 ... 161 161 161]
 [160 160 160 ... 161 161 161]
 ...
 [154 154 154 ... 156 156 156]
 [167 167 167 ... 169 169 169]
 [172 172 172 ... 174 174 174]]


In [45]:
df['Imagen con zoom'][0].shape  

(150, 150)

In [46]:
# Crear una nueva columna para almacenar solo los registros que cumplan con la forma deseada
df['Imagen Zoom Valida'] = df['Imagen con zoom'].apply(lambda x: x if np.array(x).shape == (150, 150, 3) else None)

# Eliminar los registros que no cumplen con la forma deseada
df = df.dropna(subset=['Imagen Zoom Valida'])

# Convertir la columna 'Hitos faciales válidos' en un array NumPy
puntos = np.array(df['Imagen Zoom Valida'].tolist())

# Verificar la forma de X
print(puntos.shape)

(5020, 150, 150, 3)


In [47]:
print(df['Imagen'].shape)
print(df['Imagen con zoom'].shape)

(5020,)
(5020,)


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5035 entries, 0 to 5055
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Imagen                  5035 non-null   object
 1   Imagen con zoom         5035 non-null   object
 2   Hitos faciales          5035 non-null   object
 3   Etiquetas               5035 non-null   object
 4   Hitos faciales válidos  5035 non-null   object
dtypes: object(5)
memory usage: 365.1+ KB


In [8]:
# Crear una nueva columna para almacenar solo los registros que cumplan con la forma deseada
df['Hitos faciales válidos'] = df['Hitos faciales'].apply(lambda x: x if np.array(x).shape == (1, 72, 2) else None)

# Eliminar los registros que no cumplen con la forma deseada
df = df.dropna(subset=['Hitos faciales válidos'])

# Convertir la columna 'Hitos faciales válidos' en un array NumPy
puntos = np.array(df['Hitos faciales válidos'].tolist())

# Verificar la forma de X
print(puntos.shape)

(5035, 1, 72, 2)


In [48]:
X = np.array(df['Imagen con zoom'])
y = df['Etiquetas']

In [43]:
X[0].shape

(150, 150, 3)

In [49]:
etiquetas = y.unique()
y_encoded = pd.Categorical(y, categories=etiquetas).codes

In [50]:
# Dividir el dataset en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [51]:
# Convertir las etiquetas a formato one-hot
y_train_one_hot = to_categorical(y_train)
y_val_one_hot = to_categorical(y_val)

In [52]:

# Convertir las imágenes a arrays numpy y normalizarlas
X_train = np.array([np.array(img) for img in X_train])
X_val = np.array([np.array(img) for img in X_val])


In [53]:
# Normalizar las imágenes dividiendo por 255.0
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0

In [54]:
# Definir el generador de imágenes para aumentar el dataset de entrenamiento
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [55]:
# Aumentar el dataset de entrenamiento utilizando el generador de imágenes
train_generator = datagen.flow(X_train, y_train_one_hot, batch_size=32)


In [56]:
# Definir el modelo de red neuronal convolucional (CNN)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(etiquetas), activation='softmax')  # Capa de salida con activación softmax
])


C:\Users\Acer Nitro 5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [57]:
# Compilar el modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [58]:
# Entrenar el modelo con el callback de TensorBoard
history = model.fit(train_generator,
                    epochs=5,
                    validation_data=(X_val, y_val_one_hot),
                    verbose=1,
                    )


Epoch 1/5


C:\Users\Acer Nitro 5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


126/126 ━━━━━━━━━━━━━━━━━━━━ 119s 703ms/step - accuracy: 0.4220 - loss: 1.4231 - val_accuracy: 0.6245 - val_loss: 0.8746
Epoch 2/5
126/126 ━━━━━━━━━━━━━━━━━━━━ 87s 684ms/step - accuracy: 0.5864 - loss: 0.9326 - val_accuracy: 0.6404 - val_loss: 0.7915
Epoch 3/5
126/126 ━━━━━━━━━━━━━━━━━━━━ 95s 743ms/step - accuracy: 0.5916 - loss: 0.8944 - val_accuracy: 0.6633 - val_loss: 0.8261
Epoch 4/5
126/126 ━━━━━━━━━━━━━━━━━━━━ 93s 726ms/step - accuracy: 0.6237 - loss: 0.8649 - val_accuracy: 0.6554 - val_loss: 0.7751
Epoch 5/5
126/126 ━━━━━━━━━━━━━━━━━━━━ 93s 727ms/step - accuracy: 0.6084 - loss: 0.8784 - val_accuracy: 0.6653 - val_loss: 0.7821


In [59]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_val, y_val_one_hot, verbose=0)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Loss: 0.7821480631828308, Accuracy: 0.6653386354446411


In [60]:
model.save('modeloCNN.h5')